## 1. Carga y Limpieza de Datos de Crímenes

En esta sección cargamos los datos de delitos, realizamos una limpieza inicial y filtramos únicamente los robos a transeúntes en vía pública con y sin violencia.

In [ ]:
from functools import reduce
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely.geometry as sg
import seaborn as sns
import numpy as np

crimen = pd.read_csv('delitos.csv', skiprows=1)
crimen.replace('GUSTAVO A MADERO', 'GUSTAVO A. MADERO', inplace=True)
print(f"Dimensiones del DataFrame: {crimen.shape}")

print("Columnas del DataFrame:", crimen.columns.tolist())
print("\nConteo de tipos de delitos:")
print(crimen['DELITO'].value_counts())

crimen = crimen[crimen['DELITO'] == 'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA']
print(f"Dimensiones después del filtrado: {crimen.shape}")
robos_alcaldia = crimen.groupby('ALCALDÍA HECHOS')['DELITO'].count()
print("Robos por Alcaldía:")
print(robos_alcaldia)


## 2. Carga y Procesamiento de Datos de Población

Se cargan los datos de población, se agrupan por alcaldía y se combinan con los datos de robos para calcular tasas por 100,000 habitantes.

In [ ]:
poblacion = pd.read_csv('poblacion.csv')
poblacion_alcaldia = poblacion.groupby('alcaldia')['poblacion'].sum()
print("Población por Alcaldía:")
print(poblacion_alcaldia)

robos_poblacion_df = pd.DataFrame({
    'robos': robos_alcaldia,
    'poblacion': poblacion_alcaldia
}).dropna()

robos_poblacion_df['tasa_robos'] = (robos_poblacion_df['robos']/robos_poblacion_df['poblacion'])*100000
robos_poblacion_df


## 3. Filtrado de Alcaldías de Interés y Limpieza de Coordenadas

Se enfocan los datos en las alcaldías de interés y se asegura la calidad geoespacial al eliminar registros sin coordenadas.


In [ ]:
alcaldias_interes = ['CUAUHTEMOC', 'GUSTAVO A. MADERO', 'CUAJIMALPA DE MORELOS']
crimen = crimen[crimen['ALCALDÍA HECHOS'].isin(alcaldias_interes)]
crimen['COORD X'] = pd.to_numeric(crimen['COORD X'], errors='coerce')
crimen['COORD Y'] = pd.to_numeric(crimen['COORD Y'], errors='coerce')
crimen = crimen.dropna(subset=['COORD X','COORD Y'])

robos_alcaldia = crimen.groupby('ALCALDÍA HECHOS')['DELITO'].count()
print("Robos por Alcaldía después del filtrado:")
print(robos_alcaldia)

poblacion = poblacion[poblacion['alcaldia'].isin(alcaldias_interes)]
poblacion_alcaldia = poblacion.groupby('alcaldia')['poblacion'].sum()
print("Población por Alcaldía después del filtrado:")
print(poblacion_alcaldia)

## 4. Análisis Geoespacial

Se genera un mapa base, se incorporan los datos de crímenes como puntos georreferenciados y se filtra el mapa a las alcaldías de interés.

In [ ]:
# Ruta al archivo GeoJSON
ruta_geojson = 'mapa.json'

# Carga del mapa base
agebs = gpd.read_file(ruta_geojson)
# Creación de geometrías a partir de coordenadas
crimen['geometry'] = crimen.apply(lambda row: sg.Point(row['COORD X'], row['COORD Y']), axis=1)
crimen_gdf = gpd.GeoDataFrame(crimen, geometry='geometry')

# Asignación del CRS (Sistema de Referencia de Coordenadas)
crimen_gdf.set_crs(agebs.crs, inplace=True)
print("CRS del mapa base:", agebs.crs)
print("CRS de crímenes:", crimen_gdf.crs)

In [ ]:
# Configuración de la figura
fig, ax = plt.subplots(figsize=(15, 15))  # Tamaño aumentado para mejor visualización

# Plot del mapa base
agebs.plot(ax=ax, color='lightgrey', edgecolor='black')

# Plot de los crímenes
crimen_gdf.plot(ax=ax, markersize=1, color='red', alpha=1, marker='o', label='Crímenes')

# Ajuste de los límites del mapa con un margen
xmin, ymin, xmax, ymax = agebs.total_bounds
margin_x = (xmax - xmin) * 0.05
margin_y = (ymax - ymin) * 0.05
ax.set_xlim(xmin - margin_x, xmax + margin_x)
ax.set_ylim(ymin - margin_y, ymax + margin_y)

# Añadir leyenda y título
plt.legend()
plt.title('Mapa de Áreas y Crímenes')
plt.show()

## 5. Análisis de Variables Independientes

Se incorporan variables como densidad de población, grado de escolaridad, número de escuelas y uso de suelo no residencial. Estas variables pueden influir en la incidencia delictiva.


### 5.1. Densidad de Población

Se calcula la densidad de población (habitantes por km²) para entender su relación con los robos.


In [ ]:
agebs['area_km2'] = agebs['AREA']/100
poblacion_alcaldia_df = poblacion_alcaldia.reset_index()
poblacion_alcaldia_df.columns = ['NOMGEO','poblacion']
densidad_df = agebs[['NOMGEO','area_km2']].merge(poblacion_alcaldia_df, on='NOMGEO')
densidad_df['densidad_poblacion'] = densidad_df['poblacion']/densidad_df['area_km2']
agebs = agebs.merge(densidad_df[['NOMGEO','densidad_poblacion']], on='NOMGEO')

fig, ax = plt.subplots(figsize=(10,10))
agebs.plot(column='densidad_poblacion', cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
plt.title('Densidad de Población por Alcaldía (2010)')
plt.axis('off')
plt.show()

plt.figure(figsize=(8,6))
sns.barplot(x='NOMGEO', y='densidad_poblacion', data=densidad_df, palette='Blues_d')
plt.xlabel('Alcaldía')
plt.ylabel('Densidad de Población (hab/km²)')
plt.title('Densidad de Población por Alcaldía (2010)')
plt.show()

### 5.2. Escolaridad

Se analiza el grado de escolaridad promedio en las alcaldías de interés para 2020.

https://planeacion.sep.gob.mx/Doc/Atlas_estados/CDMX.pdf

In [ ]:
escolaridad = pd.read_csv('escolaridad.csv')
escolaridad = escolaridad[['Municipio', 'Grado de escolaridad promedio']]
escolaridad = escolaridad[escolaridad['Municipio'].isin(alcaldias_interes)]

plt.figure(figsize=(8,6))
sns.barplot(x='Municipio', y='Grado de escolaridad promedio', data=escolaridad, palette='Blues_d')
plt.xlabel('Alcaldía')
plt.ylabel('Grado de Escolaridad Promedio (años)')
plt.title('Grado de Escolaridad Promedio por Alcaldía (2020)')
plt.show()

escolaridad = escolaridad.rename(columns={'Municipio':'NOMGEO'})
agebs = agebs.merge(escolaridad, on='NOMGEO')

fig, ax = plt.subplots(figsize=(10,10))
agebs.plot(column='Grado de escolaridad promedio', cmap='Greens', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
plt.title('Grado de Escolaridad Promedio por Alcaldía (2020)')
plt.axis('off')
plt.show()


### 5.3. Escuelas Públicas y Privadas

Se calcula la cantidad de escuelas públicas y privadas por alcaldía y se analiza su densidad.

https://datos.cdmx.gob.mx/dataset/escuelas-publicas
https://datos.cdmx.gob.mx/dataset/escuelas-privadas

In [ ]:
escuelas_privadas = pd.read_csv('escuelas_privadas.csv')
escuelas_publicas = pd.read_csv('escuelas_publicas.csv')

escuelas_privadas = escuelas_privadas[escuelas_privadas['alcaldia'].isin(alcaldias_interes)]
escuelas_publicas = escuelas_publicas[escuelas_publicas['alcaldia'].isin(alcaldias_interes)]

conteo_privadas = escuelas_privadas.groupby('alcaldia').size().reset_index(name='escuelas_privadas')
conteo_publicas = escuelas_publicas.groupby('alcaldia').size().reset_index(name='escuelas_publicas')
conteo_total_escuelas = pd.merge(conteo_privadas, conteo_publicas, on='alcaldia', how='outer').fillna(0)
conteo_total_escuelas['escuelas_privadas'] = conteo_total_escuelas['escuelas_privadas'].astype(int)
conteo_total_escuelas['escuelas_publicas'] = conteo_total_escuelas['escuelas_publicas'].astype(int)

fig, ax = plt.subplots(figsize=(10,6))
bar_width = 0.35
index = np.arange(len(conteo_total_escuelas))
ax.bar(index, conteo_total_escuelas['escuelas_privadas'], bar_width, color='skyblue', label='Privadas')
ax.bar(index+bar_width, conteo_total_escuelas['escuelas_publicas'], bar_width, color='salmon', label='Públicas')
ax.set_xticks(index+bar_width/2)
ax.set_xticklabels(conteo_total_escuelas['alcaldia'])
ax.set_xlabel('Alcaldía')
ax.set_ylabel('Cantidad de Escuelas')
ax.set_title('Cantidad de Escuelas Privadas y Públicas por Alcaldía')
ax.legend()
plt.show()

conteo_total_escuelas = conteo_total_escuelas.rename(columns={'alcaldia':'NOMGEO'})
conteo_total_escuelas = conteo_total_escuelas.merge(agebs, on='NOMGEO')
conteo_total_escuelas = conteo_total_escuelas[['NOMGEO','escuelas_privadas','escuelas_publicas','area_km2']]
conteo_total_escuelas['densidad_escuelas_privadas'] = conteo_total_escuelas['escuelas_privadas']/conteo_total_escuelas['area_km2']
conteo_total_escuelas['densidad_escuelas_publicas'] = conteo_total_escuelas['escuelas_publicas']/conteo_total_escuelas['area_km2']
conteo_total_escuelas = conteo_total_escuelas[['NOMGEO','densidad_escuelas_privadas','densidad_escuelas_publicas']]
conteo_total_escuelas


### 5.4. Uso de Suelo No Residencial

Se analizan tipos de uso de suelo no residencial, principalmente orientados a comercio y servicios.



In [ ]:
dfUsoDeSuelo = pd.read_csv('usoDeSuelo.csv')
dfUsoDeSuelo = dfUsoDeSuelo[dfUsoDeSuelo['alcaldia'].isin(alcaldias_interes)]
dfUsoDeSuelo = dfUsoDeSuelo.groupby('alcaldia').size().reset_index(name='usoNoResidencial')

plt.figure(figsize=(8,6))
sns.barplot(x='alcaldia', y='usoNoResidencial', data=dfUsoDeSuelo)
plt.xlabel('Alcaldía')
plt.ylabel('Cantidad de Usos No Residenciales')
plt.title('Uso del Suelo No Residencial por Alcaldía')
plt.show()

In [ ]:
dfUsoDeSuelo = dfUsoDeSuelo.merge(poblacion_alcaldia.reset_index(), on='alcaldia', how='left')
dfUsoDeSuelo.head()

In [ ]:
dfUsoDeSuelo['usoNoResidencial_100k'] = (dfUsoDeSuelo['usoNoResidencial'] / dfUsoDeSuelo['poblacion']) * 100000
dfUsoDeSuelo = dfUsoDeSuelo[['alcaldia', 'usoNoResidencial_100k']]
dfUsoDeSuelo

### 5.5. Cámaras de Vigilancia (C5I)

Se considera la distribución de cámaras de vigilancia en las alcaldías de interés.

https://www.infocdmx.org.mx/index.php/2-boletines/5567-dcs-089-17.html


In [ ]:
dfCamaras = pd.read_csv('camarasC5I.csv')
dfCamaras = dfCamaras[dfCamaras['Delegación'].isin(alcaldias_interes)]
# dfCamaras = pd.merge(dfCamaras, poblacion_alcaldia, left_on='Delegación', right_on='alcaldia')
# dfCamaras['densidadCamaras'] = 
dfCamaras = dfCamaras.rename(columns = {"Delegación": "alcaldia", "Total Cámaras": "totalCamaras"})
dfCamaras.head()

In [ ]:
dfCamaras = dfCamaras.merge(poblacion_alcaldia.reset_index(), on='alcaldia', how='left')
dfCamaras['camaras_100k'] = (dfCamaras['totalCamaras']/dfCamaras['poblacion'])*100000
dfCamaras = dfCamaras[['alcaldia','camaras_100k']]
dfCamaras

# Union datos

In [ ]:
robos_poblacion_df = robos_poblacion_df.reset_index()
robos_poblacion_df = robos_poblacion_df[robos_poblacion_df['index'].isin(alcaldias_interes)]
robos_poblacion_df = robos_poblacion_df[['index', 'tasa_robos']]
robos_poblacion_df = robos_poblacion_df.rename(columns = {"index": "alcaldia"})
robos_poblacion_df.head()

In [ ]:
agebs = agebs[['NOMGEO', 'densidad_poblacion']]
agebs = agebs.rename(columns = {"NOMGEO": "alcaldia"})
agebs

In [ ]:
escolaridad = escolaridad.rename(columns = {"NOMGEO": "alcaldia", "Grado de escolaridad promedio": "escolaridad"})
escolaridad

In [ ]:
conteo_total_escuelas = conteo_total_escuelas.rename(columns = {"NOMGEO": "alcaldia"})
conteo_total_escuelas

In [ ]:
dfUsoDeSuelo

In [ ]:
dfCamaras = dfCamaras.rename(columns = {"Delegación": "alcaldia", "Total Cámaras": "totalCamaras"})
dfCamaras

In [ ]:
dfs = [robos_poblacion_df, agebs, escolaridad, conteo_total_escuelas, dfUsoDeSuelo, dfCamaras]
df = reduce(lambda left, right: pd.merge(left, right, on='alcaldia', how='outer'), dfs)
print(df.columns)
df

# Correlacion

In [ ]:
df_numerico = df.drop(columns=['alcaldia'])
correlaciones = df_numerico.corr()
correlacion_tasa_robos = correlaciones['tasa_robos']
correlacion_tasa_robos

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(df_numerico.corr(), annot=True, cmap="coolwarm", fmt=".2f", cbar=True)
plt.title("Mapa de Calor de Correlaciones")
plt.show()

In [ ]:
# Ordenar las correlaciones con 'tasa_robos'
correlacion_tasa_robos_sorted = correlacion_tasa_robos.drop('tasa_robos').sort_values()

# Crear un gráfico de barras
plt.figure(figsize=(8, 6))
correlacion_tasa_robos_sorted.plot(kind='barh', color='skyblue')
plt.title("Correlaciones con 'tasa_robos'")
plt.xlabel("Coeficiente de Correlación")
plt.ylabel("Variables")
plt.axvline(0, color='gray', linestyle='--')
plt.show()


In [ ]:
covarianza = df_numerico.cov()

plt.figure(figsize=(10, 8))
sns.heatmap(covarianza, annot=True, fmt=".2f", cmap="Blues", cbar=True)
plt.title("Mapa de Calor de Covarianzas")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Variables dependiente e independientes
X = df_numerico.drop(columns=['tasa_robos'])  # Variables independientes
y = df_numerico['tasa_robos']  # Variable dependiente

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)

# Evaluación del modelo
mse = mean_squared_error(y_test, y_pred)  # Error cuadrático medio
r2 = r2_score(y_test, y_pred)  # Coeficiente R²

print("Coeficientes:", modelo.coef_)
print("Intercepto:", modelo.intercept_)
print("MSE:", mse)
print("R²:", r2)